In [60]:
from pkg_resources import resource_filename
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import chess
matplotlib.style.use('ggplot')

In [57]:
# Import stockfish values and fens
relative_filepath = 'data/extracted_data/'
filepath = resource_filename('guerilla', relative_filepath + 'sf_values_rnd.nsv')
sf = pd.read_csv(filepath, names=['values_rnd'], header=None)

filepath = resource_filename('guerilla', relative_filepath + 'fens_rnd.nsv')
sf['fens_rnd'] = pd.read_csv(filepath, header=None)

filepath = resource_filename('guerilla', relative_filepath + 'sf_values.nsv')
sf['values'] = pd.read_csv(filepath, header=None)

filepath = resource_filename('guerilla', relative_filepath + 'fens.nsv')
sf['fens'] = pd.read_csv(filepath, header=None)

87798

In [63]:
# Add numbering 
start_fen = 'rnbqkbnr/pppppppp/8/8/8/8/PPPPPPPP/RNBQKBNR w KQkq - 0 1'
sf.iloc[sf['fens'] == start_fen]['move_count'] = 0

NotImplementedError: iLocation based boolean indexing on an integer type is not available

In [28]:
#values.plot.hist(bins=100)
hist, bins = np.histogram(sf['values'], bins = 100)
plt.bar(bins[:-1], hist*1.0 / hist.sum(), width=(bins[1]-bins[0]))
plt.xticks(np.arange(0, 1.1, 0.1))
#plt.xlim([-2000, 2000])
plt.title('Stockfish Score Distribution')
plt.xlabel('P(Win)')
plt.ylabel('% of fens')
plt.show()

In [31]:
sf[['values','values_rnd']].plot.hist(bins=100, stacked = False, alpha = 0.75 )
plt.show()